In [1]:
import os
import json
import numpy as np
from scipy.stats import poisson
from scipy.optimize import minimize

Poisson 4 Parâmetros

In [2]:
from modelos.poisson_4params_decay import PoissonModel

In [2]:
# Função para converter odds no formato Moneyline para probabilidade
def odds_to_probability(odd):
    if odd > 0:
        return 100 / (odd + 100)
    else:
        return abs(odd) / (abs(odd) + 100)

In [5]:
# Parâmetros e variáveis do modelo
competition = "brasileirao"  # Nome da competição
years = [2021, 2022, 2023]  # Anos a serem processados
all_predictions = {}

for year in years:
    print(f"Processando o ano {year}...")
    
    dados = f"dados/{competition}_{year}.json"

    # Instanciar o modelo para o ano específico
    model = PoissonModel(dados, year)

    # Configurar o intervalo de jogos por ano
    if year == 2021:
        model.max_games = 280  # 280 jogos para treinamento
    elif year == 2022:
        model.max_games = 760  # Continuar a contagem (280 de 2021 + 480)
    elif year == 2023:
        model.max_games = 1140  # Continuar a contagem (280 de 2021 + 280 de 2022 + 280 de 2023)

    # Treinar o modelo com os jogos do ano atual
    model.run_model(verbose=True)

    
    
    # Carregar todos os jogos do ano
    with open(f"dados/{competition}_{year}.json", "r") as f:
        all_games = json.load(f)
    
    # Filtrar os 100 jogos de teste (do jogo 281 ao 380)
    # test_games = {k: v for k, v in all_games.items() if 280 < int(k) <= 380}

    # Ajustar os 100 jogos de teste corretamente
    if year == 2021:
        test_games = {k: v for k, v in all_games.items() if 280 < int(k) <= 380}
    elif year == 2022:
        test_games = {k: v for k, v in all_games.items() if 760 < int(k) <= 860}
    elif year == 2023:
        test_games = {k: v for k, v in all_games.items() if 1140 < int(k) <= 1240}
    
    # Obter parâmetros otimizados
    success, inx, parameters = model.optimize_parameters(verbose=True)
    
    # Prever probabilidades se a otimização foi bem-sucedida
    if success:
        for game_id, game_data in test_games.items():
            home_team = game_data["Home"]
            away_team = game_data["Away"]
            
            # Prever as probabilidades de vitória, empate e derrota usando o modelo
            home_win_prob, draw_prob, away_win_prob = model.predict_match_probability(home_team, away_team, inx)

            # Normalizar as probabilidades
            total_prob = home_win_prob + draw_prob + away_win_prob

            home_win_prob /= total_prob
            draw_prob /= total_prob
            away_win_prob /= total_prob
            
            # Calcular as probabilidades das odds
            odd_home = game_data.get("Odd home", 0)  # Obtém a odd da vitória do time da casa
            odd_draw = game_data.get("Odd X", 0)  # Obtém a odd do empate
            odd_away = game_data.get("Odd away", 0)  # Obtém a odd da vitória do visitante
            
            # Converter odds em probabilidades
            prob_home_odd = odds_to_probability(odd_home)
            prob_draw_odd = odds_to_probability(odd_draw)
            prob_away_odd = odds_to_probability(odd_away)

            # Normalizar probabilidades para garantir soma = 1
            total_prob = prob_home_odd + prob_draw_odd + prob_away_odd

            prob_home_odd /= total_prob
            prob_draw_odd /= total_prob
            prob_away_odd /= total_prob
            
            # Salvar probabilidades no dicionário principal
            all_predictions[f"{year}_{game_id}"] = {
                "ModelProbabilities": {
                    "HomeWin": home_win_prob,
                    "Draw": draw_prob,
                    "AwayWin": away_win_prob
                },
                "OddsProbabilities": {
                    "HomeWin": prob_home_odd,
                    "Draw": prob_draw_odd,
                    "AwayWin": prob_away_odd
                }
            }

# Salvar todas as previsões e probabilidades das odds em um único JSON
with open("probabilities.json", "w") as f:
    json.dump(all_predictions, f, indent=4)

print("Arquivo 'probabilities.json' criado com as probabilidades do modelo e das odds.")


Processando o ano 2021...
Parâmetros estimados com sucesso!
Processando o ano 2022...
Parâmetros estimados com sucesso!
Processando o ano 2023...
Parâmetros estimados com sucesso!
Arquivo 'probabilities.json' criado com as probabilidades do modelo e das odds.


Poisson 2 Parâmetros

In [3]:
from modelos.poisson_2params_home_away_decay import PoissonModel

In [4]:
# Parâmetros e variáveis do modelo
competition = "brasileirao"  # Nome da competição
years = [2021, 2022, 2023]  # Anos a serem processados
all_predictions = {}

for year in years:
    print(f"Processando o ano {year}...")
    
    dados = f"dados/{competition}_{year}.json"

    # Instanciar o modelo para o ano específico
    model = PoissonModel(dados, year)

    # Configurar o intervalo de jogos por ano
    if year == 2021:
        model.max_games = 280  # 280 jogos para treinamento
    elif year == 2022:
        model.max_games = 760  # Continuar a contagem (280 de 2021 + 480)
    elif year == 2023:
        model.max_games = 1140  # Continuar a contagem (280 de 2021 + 280 de 2022 + 280 de 2023)

    # Treinar o modelo com os jogos do ano atual
    model.run_model(verbose=True)

    
    
    # Carregar todos os jogos do ano
    with open(f"dados/{competition}_{year}.json", "r") as f:
        all_games = json.load(f)
    
    # Filtrar os 100 jogos de teste (do jogo 281 ao 380)
    # test_games = {k: v for k, v in all_games.items() if 280 < int(k) <= 380}

    # Ajustar os 100 jogos de teste corretamente
    if year == 2021:
        test_games = {k: v for k, v in all_games.items() if 280 < int(k) <= 380}
    elif year == 2022:
        test_games = {k: v for k, v in all_games.items() if 760 < int(k) <= 860}
    elif year == 2023:
        test_games = {k: v for k, v in all_games.items() if 1140 < int(k) <= 1240}
    
    # Obter parâmetros otimizados
    success, inx, gamma, parameters = model.optimize_parameters(verbose=True)
    
    # Prever probabilidades se a otimização foi bem-sucedida
    if success:
        for game_id, game_data in test_games.items():
            home_team = game_data["Home"]
            away_team = game_data["Away"]
            
            # Prever as probabilidades de vitória, empate e derrota usando o modelo
            home_win_prob, draw_prob, away_win_prob = model.predict_match_probability(home_team, away_team, inx, gamma)

            # Normalizar as probabilidades
            total_prob = home_win_prob + draw_prob + away_win_prob

            home_win_prob /= total_prob
            draw_prob /= total_prob
            away_win_prob /= total_prob
            
            # Calcular as probabilidades das odds
            odd_home = game_data.get("Odd home", 0)  # Obtém a odd da vitória do time da casa
            odd_draw = game_data.get("Odd X", 0)  # Obtém a odd do empate
            odd_away = game_data.get("Odd away", 0)  # Obtém a odd da vitória do visitante
            
            # Converter odds em probabilidades
            prob_home_odd = odds_to_probability(odd_home)
            prob_draw_odd = odds_to_probability(odd_draw)
            prob_away_odd = odds_to_probability(odd_away)

            # Normalizar probabilidades para garantir soma = 1
            total_prob = prob_home_odd + prob_draw_odd + prob_away_odd

            prob_home_odd /= total_prob
            prob_draw_odd /= total_prob
            prob_away_odd /= total_prob
            
            # Salvar probabilidades no dicionário principal
            all_predictions[f"{year}_{game_id}"] = {
                "ModelProbabilities": {
                    "HomeWin": home_win_prob,
                    "Draw": draw_prob,
                    "AwayWin": away_win_prob
                },
                "OddsProbabilities": {
                    "HomeWin": prob_home_odd,
                    "Draw": prob_draw_odd,
                    "AwayWin": prob_away_odd
                }
            }

# Salvar todas as previsões e probabilidades das odds em um único JSON
with open("probabilities2param.json", "w") as f:
    json.dump(all_predictions, f, indent=4)

print("Arquivo 'probabilities2param.json' criado com as probabilidades do modelo e das odds.")


Processando o ano 2021...
Parâmetros estimados com sucesso!
Processando o ano 2022...
Parâmetros estimados com sucesso!
Processando o ano 2023...
Parâmetros estimados com sucesso!
Arquivo 'probabilities2param.json' criado com as probabilidades do modelo e das odds.
